# load dataset

In [1]:
import os
import torch
import torchio as tio

dataset = torch.load('../data/preproc.pt', weights_only=False)

# split into train and test
BATCH_SIZE = 10

train_data, test_data = torch.utils.data.random_split(dataset, [0.7, 0.3])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# check loaders
for i, batch in enumerate(train_loader):
    inputs = batch['mri'][tio.DATA] # shape: (B, 1, D, H, W)
    labels = batch['label']
    print(i, inputs.shape, labels.shape)
    break

train_N = len(train_loader.dataset)
test_N = len(test_loader.dataset)
len(dataset), train_N, test_N

/home/jstabet/miniforge3/envs/dicom/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

0 torch.Size([10, 1, 512, 512, 60]) torch.Size([10])


(290, 203, 87)

# model

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class MRIClassifier(nn.Module):
    def __init__(self, in_channels=1, num_classes=2):
        super(MRIClassifier, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.pool1 = nn.MaxPool3d(kernel_size=2)

        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.pool2 = nn.MaxPool3d(kernel_size=2)

        self.global_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.global_pool(x) # shape: (B, 64, 1, 1, 1)
        x = x.view(x.size(0), -1) # shape: (B, 64)
        return self.fc(x)

# train

In [ ]:
# compile model
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MRIClassifier()
model = nn.DataParallel(model)
model.to(DEVICE)

# # weighted loss
# import numpy as np
# from sklearn.utils.class_weight import compute_class_weight
# train_labels = np.array([dataset[i][1] for i in train_data.indices])
# classes = np.arange(train_loader.dataset.dataset.num_classes)
# class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_labels)
# loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32)).to(DEVICE) # expects logits

loss_fn = torch.nn.CrossEntropyLoss() # expects logits
optimizer = torch.optim.Adam(model.parameters())

# train
EPOCHS = 100
train_losses = []
val_losses = []
train_accs = []
val_accs = []
best_val_loss = torch.inf
for epoch in range(EPOCHS):
    # train
    model.train()
    running_loss = 0.0
    running_correct = 0
    for i, batch in enumerate(train_loader):
        inputs = batch['mri'][tio.DATA] # shape: (B, 1, D, H, W)
        labels = batch['label']
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_correct += (torch.softmax(outputs, dim=1).argmax(dim=1) == labels).sum().item()

    print(f"Epoch {epoch+1}/{EPOCHS}\tTrain Loss: {running_loss/train_N:.4f} | Train Acc: {running_correct/train_N:.4f}")
    train_losses.append(running_loss/train_N)
    train_accs.append(running_correct/train_N)

    # save model
    torch.save(model, f'../results/model.pt')

Epoch 1/100	Train Loss: 0.0699 | Train Acc: 0.6108


Epoch 2/100	Train Loss: 0.0542 | Train Acc: 0.7980


Epoch 3/100	Train Loss: 0.0515 | Train Acc: 0.7980


Epoch 4/100	Train Loss: 0.0512 | Train Acc: 0.7980


Epoch 5/100	Train Loss: 0.0508 | Train Acc: 0.7980


...

Epoch 97/100	Train Loss: 0.0546 | Train Acc: 0.7980


Epoch 98/100	Train Loss: 0.0513 | Train Acc: 0.7980


Epoch 99/100	Train Loss: 0.0525 | Train Acc: 0.7980


Epoch 100/100	Train Loss: 0.0525 | Train Acc: 0.7980


AttributeError: 'str' object has no attribute 'to'

# test

In [ ]:
import numpy as np

# load best model
model = torch.load(f'../results/model.pt', weights_only=False).to(DEVICE)

model.eval()

y_true = []
y_logits = []
y_scores = []
y_pred = []

correct = 0

with torch.no_grad():
    for i, batch in enumerate(test_loader):
        inputs = batch['mri'][tio.DATA] # shape: (B, 1, D, H, W)
        labels = batch['label']
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)

        y_true.extend(labels.cpu().numpy())
        y_logits.extend(outputs.cpu().numpy())
        y_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())
        y_pred.extend(torch.softmax(outputs, dim=1).argmax(dim=1).cpu().numpy())

        correct += (torch.softmax(outputs, dim=1).argmax(dim=1) == labels).sum().item()

y_true = np.array(y_true)
y_logits = np.array(y_logits)
y_scores = np.array(y_scores)
y_pred = np.array(y_pred)

print()
print(f'Test Accuracy: {correct/test_N:.4f}')

# save

In [ ]:
import pandas as pd

# save datasets, optimizer state, and metrics to metadata file
torch.save({'train_loader': train_loader,#train_data,
            'test_loader': test_loader,#test_data,
            'optimizer_state': optimizer.state_dict(),
            'y_true': y_true,
            'y_logits': y_logits,
            'y_scores': y_scores,
            'y_pred': y_pred,
            'metrics': pd.DataFrame({'train_loss': train_losses,
                                     'train_acc': train_accs,})},
            
            f'../results/metadata.pt')